In [ ]:
# Soligence App
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import numpy as np

# Load cryptocurrency price data
data = pd.read_csv('combined_crypto_data.csv')

def home_section():
    st.header("SOLiGence")
    st.write(
        "Solent Intelligence (SOLiGence) is a leading financial multinational organisation that deals"
        " with stock and shares, saving and investments.")

    # Streamlit app
    st.title('Welcome to SOLiGence')
    st.write("Your Intelligent Coin Trading Platform")

    # Brief Description
    st.write("Empower your cryptocurrency trading decisions with AI-driven insights and real-time data.")

    # Call to Action (CTA) Button
    if st.button("Get Started"):
        st.write("Let's explore the world of cryptocurrency trading together!")

        # Background Image
        st.image('sal.jpg', use_column_width=True)
        if 'section' not in st.session_state:
            st.session_state.section = "Home"

        # News and Updates
        st.write("News and Updates:")
        st.info("Stay tuned for the latest updates and trends in the cryptocurrency market!")

        # Testimonials
        st.write("What Our Users Say:")
        st.write("The SOLiGence app transformed how I approach cryptocurrency trading. Highly recommended!")

        # Contact Information
        st.write("Contact Us:")
        st.write("For inquiries, email us at info@soligence.com")

        # Social Media Links
        st.write("Connect with Us:")
        st.markdown("[Twitter](https://twitter.com/SOLiGenceApp) [LinkedIn](https://linkedin.com/company/soligence)")

        # Privacy and Disclaimer
        st.write("Privacy Policy | Disclaimer")

# Streamlit app
st.title('Welcome to SOLiGence')

def about_us():
    st.title("About Solent Intelligence Ltd.")
    st.write(
        "The scale of this organization's operation is impressive, with millions of subscribers and over 150 billion "
        "pounds worth of investments. This emphasizes the substantial influence that data-driven decisions can have "
        "on managing such a significant amount of assets. The app's focus on implementing an Intelligent Coin Trading "
        "(IST) platform, specifically tailored for crypto coin predictions, resonates deeply with me. The idea of "
        "utilizing AI to predict cryptocurrency prices on different timeframes, such as daily, weekly, monthly, "
        "and quarterly, truly piques my interest. This approach aligns perfectly with my desire to explore how AI can "
        "shape and enhance our daily lives. The app's ability to recommend trading opportunities by analyzing "
        "AI-generated predictions showcases the tangible applications of data science in the financial world. "
        "Considering a more neutral perspective, while the concept of the app is exciting, there are potential "
        "challenges that need to be acknowledged. Cryptocurrency markets are known for their volatility, and even the "
        "most sophisticated AI predictions might not always be entirely accurate. Users relying solely on these "
        "predictions could face risks if market conditions change unexpectedly.")

def dataset():
    st.header("Crypto Dataset of Five Coins")

    # Sidebar options
    st.sidebar.subheader("Dataset Options")

    # Sorting
    sort_column = st.sidebar.multiselect("Sort by:", data.columns)
    ascending = st.sidebar.checkbox("Ascending")
    sorted_data = data.sort_values(by=sort_column, ascending=ascending)

    # Filtering
    selected_crypto = st.sidebar.selectbox("Filter by cryptocurrency:", ["All"] + data['Crypto'].unique())
    if "All" not in selected_crypto:
        sorted_data = sorted_data[sorted_data['Crypto'].isin(selected_crypto)]

    # Pagination
    page_size = st.sidebar.number_input("Items per page:", min_value=1, value=10)
    page_number = st.sidebar.number_input("Page number:", min_value=1, value=1)
    start_idx = (page_number - 1) * page_size
    end_idx = start_idx + page_size
    paginated_data = sorted_data.iloc[start_idx:end_idx]

    # Display the dataset using a dataframe
    st.subheader("Dataset Overview")
    st.dataframe(paginated_data)

    # Provide an option to show a table view
    show_table = st.checkbox("Show as Table")

    # Display the dataset as a table if the checkbox is selected
    if show_table:
        st.subheader("Dataset Table View")
        st.table(paginated_data)
        
def analyze_coin_correlation(data):
    """
    Streamlit interface for analyzing and displaying the top four positively and 
    negatively correlated cryptocurrencies with user-selected coins.
    """
    st.header("Coin Correlation Analysis")
    st.write("Streamlit interface for analyzing and displaying the top four positively and negatively correlated cryptocurrencies with user-selected coins.")

    coins_list = data.columns.tolist()
    st.write("Available coins:", ', '.join(coins_list))

    coins_selected = st.multiselect("Select the coins you want to analyze:", coins_list)

    for coin_selected in coins_selected:
        st.subheader(f"Analysis for {coin_selected}")
        selected_coin_prices = data[coin_selected]
        correlations = data.corrwith(selected_coin_prices)
        sorted_correlations = correlations.sort_values(ascending=False).drop(coin_selected)
        positively_correlated = sorted_correlations.head(4)
        negatively_correlated = sorted_correlations.tail(4)

        st.write(f"Top four positively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(positively_correlated)
        st.write(f"Top four negatively correlated cryptocurrencies with {coin_selected}:")
        st.dataframe(negatively_correlated)

        
def plot_moving_average(crypto_data):
    """
    Plots the moving average for a user-selected cryptocurrency.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data.
    """
    st.header("Moving Average Analysis")
    st.write("Plots the moving average for a user-selected cryptocurrency.")

    # Get the list of available cryptocurrencies
    available_coins = crypto_data['Crypto'].unique()

    # Streamlit user input for selecting the coin
    coin_selected = st.selectbox("Select the cryptocurrency you want to analyze:", available_coins)

    # User input for selecting the window size
    window_size = st.radio("Select the window size for the moving average:", ['Short (30-day MA)', 'Medium (60-day MA)', 'Long (90-day MA)'])

    # Map window size selection to the corresponding window value
    window_mapping = {'Short (30-day MA)': 30, 'Medium (60-day MA)': 60, 'Long (90-day MA)': 90}
    window = window_mapping[window_size]

    # Filter data for the selected coin
    selected_data = crypto_data[crypto_data['Crypto'] == coin_selected].copy()

    # Calculate the moving average based on the selected window size
    selected_data['MA'] = selected_data['Close'].rolling(window=window).mean()
    window_title = f'{window}-day MA'

    # Plotting
    st.write(f"Price Chart for {coin_selected} with {window_title}")
    plt.figure(figsize=(10, 6))
    plt.plot(selected_data.index, selected_data['Close'], label=f'{coin_selected} Price')
    plt.plot(selected_data.index, selected_data['MA'], label=window_title)
    plt.title(f'{coin_selected} Price and Moving Average')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    st.pyplot(plt)

        
def plot_crypto_prices_visualization(crypto_data, compare_option, coins, metrics, start_date, end_date):
    """
    Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons.

    Parameters:
    - compare_option (str): Option for comparison. Can be 'coins' to compare metrics between coins, 'metrics' to compare within a coin, or 'none' for no comparison.
    - coins (list): List of cryptocurrencies to plot.
    - metrics (list): List of price metrics to plot (e.g., 'Close', 'Open', 'High', 'Low', 'Volume').
    - start_date (str): Start date of the interval in 'YYYY-MM-DD' format.
    - end_date (str): End date of the interval in 'YYYY-MM-DD' format.
    """
    st.title('Explore historical cryptocurrency prices.')
    st.header('Cryptocurrency Price Metrics Visualization')
    st.write("Plots the price metrics of selected cryptocurrencies over a specified interval, with optional comparisons")

    # Filter data for the selected coins and interval
    selected_data = crypto_data[(crypto_data['Crypto'].isin(coins)) & (crypto_data.index >= start_date) & (crypto_data.index <= end_date)]

    # Plot the selected price metrics
    plt.figure(figsize=(12, 6))

    if compare_option == 'coins':
        for metric in metrics:
            for coin in coins:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    elif compare_option == 'metrics':
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")
    else:
        for coin in coins:
            for metric in metrics:
                coin_data = selected_data[selected_data['Crypto'] == coin]
                plt.plot(coin_data.index, coin_data[metric], label=f"{coin} {metric}")

    # Add labels and title
    plt.title('Cryptocurrency Prices Over Time')
    plt.xlabel('Date')
    plt.ylabel('Price/Volume')
    plt.xticks(rotation=45)
    plt.legend()
    plt.grid(True)

    # Show plot
    st.pyplot(plt)


def plot_candlestick_chart(coin='BTC-GBP', period='D'):
    st.subheader('Cryptocurrency Price Visualization by Interval')
    # Filter the DataFrame for the selected coin
    coin_data = crypto_data[crypto_data['Crypto'] == coin].copy()

    # Make sure the index is a DatetimeIndex
    coin_data.index = pd.to_datetime(coin_data.index)

    # Resample data based on the selected period
    if period.upper() == 'W':
        resampled_data = coin_data.resample('W').agg({'Open': 'first',
                                                      'High': 'max',
                                                      'Low': 'min',
                                                      'Close': 'last',
                                                      'Volume': 'sum'})
    elif period.upper() == 'M':
        resampled_data = coin_data.resample('M').agg({'Open': 'first',
                                                      'High': 'max',
                                                      'Low': 'min',
                                                      'Close': 'last',
                                                      'Volume': 'sum'})
    else:  # Default to daily data if period is not weekly or monthly
        resampled_data = coin_data

    # Plotting
    st.title(f'{coin} {period.upper()} Candlestick Chart')
    mpf.plot(resampled_data, type='candle', style='charles', volume=True,
             title=f'{coin} {period.upper()} Candlestick Chart', warn_too_much_data=1000)

def visualize_market_state(crypto_data):
    """
    Predicts the market state (up or down) for a group of chosen cryptocurrencies.

    Parameters:
    - crypto_data (pd.DataFrame): DataFrame containing the cryptocurrency data with date as index.

    Returns:
    - market_state (pd.Series): Series indicating the market state for each day.
    """

    # User input for selecting cryptocurrencies
    st.sidebar.title("Market State Prediction")
    available_coins = crypto_data['Crypto'].unique()
    coins_selected = st.sidebar.multiselect("Select cryptocurrencies to analyze", available_coins)

    # Filter data for the selected coins
    selected_data = crypto_data[crypto_data['Crypto'].isin(coins_selected)].copy()

    # Visualize market state
    fig, ax = plt.subplots(figsize=(10, 6))
    selected_data['Close'].plot(ax=ax, label='Close Price')
    ax.scatter(selected_data[selected_data['Market State'] == -1].index, 
               selected_data[selected_data['Market State'] == -1]['Close'], 
               c='r', label='Down Market', marker='v', alpha=0.5)
    ax.scatter(selected_data[selected_data['Market State'] == 1].index, 
               selected_data[selected_data['Market State'] == 1]['Close'], 
               c='g', label='Up Market', marker='^', alpha=0.5)
    ax.set_title('Market State Prediction')
    ax.set_xlabel('Date')
    ax.set_ylabel('Price')
    ax.legend()
    st.pyplot(fig)
    
    
    
if side_bars == "Predictions":
    st.title('Cryptocurrency Price Volatility Example')
    st.write("Cryptocurrency markets are known for their complexities and volatility.")

    # Select cryptocurrencies for prediction
    selected_crypto = st.multiselect("Select cryptocurrencies:", data['Crypto'].unique())

    # Prepare data for Linear Regression
    plt.figure(figsize=(10, 6))
    for crypto in selected_crypto:
        crypto_data = data[data['Crypto'] == crypto]
        x = np.array(crypto_data.index).reshape(-1, 1)
        y = crypto_data['Price']

        # Linear Regression model
        lr = LinearRegression()
        lr.fit(x, y)
        predicted = lr.predict(x)

        # Display predicted values and corresponding dates
        st.write(f"Predicted Values and Corresponding Dates for {crypto}:")
        prediction_df = pd.DataFrame({'Date': crypto_data['Date'], 'Predicted Price': predicted})
        st.write(prediction_df)

        # Display scatter plot with Linear Regression line
        plt.scatter(x, y, label=f'Actual Prices ({crypto})')
        plt.plot(x, predicted, label=f'Linear Regression ({crypto})')

        plt.xlabel('Days')
        plt.ylabel('Price')
        plt.title(f'Cryptocurrency Price Volatility Prediction for {", ".join(selected_crypto)}')
        plt.legend()
        st.pyplot(plt)

    # crypto_data = data[data['Crypto'] == 'Bitcoin']
    # Select cryptocurrency for prediction
    selected_crypto = st.selectbox("Select a cryptocurrency:", data['Crypto'].unique())
    crypto_data = data[data['Crypto'] == selected_crypto]

    # Prepare data for Linear Regression
    x = np.array(crypto_data.index).reshape(-1, 1)
    y = crypto_data['Price']

    # Linear Regression model
    lr = LinearRegression()
    lr.fit(x, y)
    predicted = lr.predict(x)

    # Display predicted values and corresponding dates
    st.write(f"Predicted Values and Corresponding Dates for {selected_crypto}:")
    prediction_df = pd.DataFrame({'Date': crypto_data['Date'], 'Predicted Price': predicted})
    st.write(prediction_df)

    st.write(f"Visualizing Cryptocurrency Price Prediction for {selected_crypto}:")
    # Display scatter plot with Linear Regression line
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, label='Actual Prices', color='blue')
    plt.plot(x, predicted, label='Linear Regression', color='red')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.title(f'Cryptocurrency Price Volatility Prediction for {selected_crypto}')
    plt.legend()
    st.pyplot(plt)

    # Prepare data for Linear Regression
    x = np.array(crypto_data.index).reshape(-1, 1)
    y = crypto_data['Price']

    # Linear Regression model
    lr = LinearRegression()
    lr.fit(x, y)
    predicted = lr.predict(x)

   # Display predicted values and corresponding dates
    st.write("Predicted Values and Corresponding Dates:")
    st.write(pd.DataFrame({'Date': crypto_data['Date'], 'Predicted Price': predicted}))

    st.write("Let's visualize the limitations of a Linear Regression model.")
    # Display scatter plot with Linear Regression line
    plt.figure(figsize=(10, 6))
    plt.scatter(x, y, label='Actual Prices', color='blue')
    plt.plot(x, predicted, label='Linear Regression', color='red')
    plt.xlabel('Days')
    plt.ylabel('Price')
    plt.title('Cryptocurrency Price Volatility')
    plt.legend()
    st.pyplot(plt)
    
    
    
    
    
# Streamlit app
st.title('Welcome to SOLiGence')
st.write("Your Intelligent Coin Trading Platform")

# Logo
# st.image('your_logo.png', width=200)

# Brief Description
st.write("Empower your cryptocurrency trading decisions with AI-driven insights and real-time data.")

# Call to Action (CTA) Button
if st.button("Get Started"):
    st.write("Let's explore the world of cryptocurrency trading together!")

    # Background Image
    st.image('sal.jpg', use_column_width=True)
    if 'section' not in st.session_state:
        st.session_state.section = "Home"

    # News and Updates
    st.write("News and Updates:")
    st.info("Stay tuned for the latest updates and trends in the cryptocurrency market!")

    # Testimonials
    st.write("What Our Users Say:")
    st.write("The SOLiGence app transformed how I approach cryptocurrency trading. Highly recommended!")

    # Contact Information
    st.write("Contact Us:")
    st.write("For inquiries, email us at info@soligence.com")

    # Social Media Links
    st.write("Connect with Us:")
    st.markdown("[Twitter](https://twitter.com/SOLiGenceApp) [LinkedIn](https://linkedin.com/company/soligence)")

    # Privacy and Disclaimer
    st.write("Privacy Policy | Disclaimer")

# Sidebar navigation
side_bars = st.sidebar.radio("Navigation", ["Home", "About Us", "Dataset","Coin Correlation","Moving Average", "Visualizations", "Predictions","NEWS"])

# Conditionals for sidebar navigation
if side_bars == "Home":
    home_section()
elif side_bars == "About Us":
    about_us()
elif side_bars == "Dataset":
    dataset()
elif side_bars == "Coin Correlation":
    analyze_coin_correlation(data)
elif side_bars == "Moving Average":
    plot_moving_average(crypto_data)
elif side_bars == "Visualizations":
    # Visualization section
    visualization_option = st.sidebar.radio("Select Visualization:", ['Price Metrics', 'Candlestick Chart','Market State Visualization'])

    if visualization_option == 'Price Metrics':
        # User input
        compare_option = st.sidebar.radio("Select comparison option:", ['coins', 'metrics', 'none'])
        # Load crypto data (replace this with your own data loading method)
        crypto_data = pd.read_csv('crypto_data.csv')
        coins = st.sidebar.multiselect("Select cryptocurrencies:", crypto_data['Crypto'].unique(), default=['BTC-GBP'])
        metrics = st.sidebar.multiselect("Select price metrics:", ['Close', 'Open', 'High', 'Low', 'Volume'], default=['Close'])
        start_date = st.sidebar.date_input("Select start date:", pd.to_datetime('2023-04-08'))
        end_date = st.sidebar.date_input("Select end date:", pd.to_datetime('2024-04-08'))

        # Plot the selected cryptocurrency prices
        if st.sidebar.button("Plot Cryptocurrency Prices"):
            plot_crypto_prices(crypto_data, compare_option, coins, metrics, start_date, end_date)

    elif visualization_option == 'Candlestick Chart':
        # User input for visualization
        coin_visualization = st.sidebar.selectbox("Select a coin for visualization:", crypto_data['Crypto'].unique(), index=0)
        period_visualization = st.sidebar.selectbox("Select period for visualization:", ['D', 'W', 'M'], index=0)

        # Plot the selected cryptocurrency chart
        if st.sidebar.button("Plot Cryptocurrency Chart"):
            plot_selected_crypto_chart(coin_visualization, period_visualization)
            
    elif visualization_option == 'Market State Visualization':
        visualize_market_state(crypto_data)

elif side_bars == "Predictions":
    predictions()
elif side_bars == "NEWS":
    news()
# # SOLiGence - Solent Intelligence

# Welcome to SOLiGence, a leading financial multinational organization specializing in stock and shares, savings, and investments.

# ## About Us

# At SOLiGence, we are committed to providing top-tier financial services and solutions to our clients. With our expert team and years of experience in the industry, we strive to offer the best investment opportunities, personalized advice, and comprehensive solutions to help you achieve your financial goals.

# ## Our Services

# 1. **Stock and Shares**: We offer a wide range of options for investing in the stock market. Our team of experts analyzes market trends and provides insights to help you make informed decisions.

# 2. **Savings**: We understand the importance of savings for a secure future. Our tailored savings plans ensure that you can build a financial cushion to meet unexpected expenses.

# 3. **Investments**: Whether you're a novice investor or an experienced one, we provide investment opportunities that align with your risk tolerance and financial objectives.

# ## Why Choose SOLiGence?

# - **Expertise**: Our team consists of financial experts who are well-versed in market trends and investment strategies.

# - **Personalized Approach**: We understand that every client has unique financial needs. Our solutions are customized to suit your individual goals.

# - **Transparency**: We believe in transparency in all our transactions and provide clear insights into investment performance and market developments.

# ## Contact Us

# For more information about our services or to schedule a consultation, please contact us at:

# - Website: [www.soligence.com](https://www.soligence.com)
# - Email: info@soligence.com
# - Phone: +1-123-456-7890

# Join SOLiGence today and let us help you build a brighter financial future!
